# Assignment  Practice Text classification with Naive Bayes 

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: 

__Student id(s)__ : 

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    <!--You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
-->
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with 40K kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercises</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>

<ol>
  <li>Normalize the values for "ministerie" and choose 10 ministeries to work with. </li>
  <li>Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies.
  It might be easier to use the representation and formula given in MRS section 13.4.1.</li>
  <li>On this collection, train NB text classifiers for 10 different classes with enough and interesting data.</li>
  <li>Compute for each term and each of your 10 classes its utility for that class using mutual information.</li>
  <li>For each class, show the top 10 words as in Figure 13.7 in MRS.</li>
  <li>Evaluate your classifiers using Precision, Recall and F1. (
       <br/>
      Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes.
      Thus do feature selection per class, and use for each class the top n best features for that class. 
      <br/>
  Also show the microaverage(s) for all 10 classes together.
  <br/>
  If you like you can also present this in a figure like MRS.13.8. 
  Then compute the F1 measure for the same number of terms as in that figure.</li>
<li> You have done the complete implementation by yourself. Congratulations! You can also use `scikit-learn` routines for all of this work. Do that. So follow [this text classification tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  and implement the same steps but now with your kamervragen dataset. Also use [mutual information feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) to select the K-best features, and compare the results as before.
</li>
  <li>Reflect and report briefly about your choices in this process and about the obtained results. Also reflect on the differences between the scikit learn approach and the "own implementation approach".</li>
</ol>

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>

<!--
<h2>Form of presentation</h2>
<ul>
    <li>Make slides or wikipages and have your system running (this could just be an IPython notebook with a classify function that accepts any string and classifies it.) ~~and be able to accept documents from the web.~~ </li>
    <li>Create one or two slides or wikipages for each of the sub exercises listed above.
</li>
<li>Make it clear in the heading of the slides which sub exercises you talk about.</li>
    <li>Show running code with one or two  good examples (a TP of course, but also a FP and an error-analysis is nice to show). </li>
</ul>
-->

<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  

### Assignment steps

#### 1 Normalize
* Everything to lowercase
* Remove shorthand, add stemming for shorthand (ez -> economische zaken)
* Add one smoothin

#### 2 Implementation Tf and Df
* Split data into 'test' and 'training'
* Create data structure for terms and doc frequencies
* Create NB classifier function

#### 3 Training NB
* Train NB with 10 classes

#### 4 Mutual information
* Compute the utility for the 10 classes

In [ ]:
from collections import defaultdict
from collections import Counter
import nltk
import math
import csv
from __future__ import division
import pandas as pd
import time

_names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

_ministeries = ['defensie', 'volksgezondheid', 'binnenlands', 'financ', 'justitie', 
                'econom', 'buitenlands', 'onderwijs', 'verkeer', 'social', 'landbouw']

_index_dataset = {
    'terms' : {},
    'classes': {}
}

def info_dataset(infile):
    with open(infile, 'r') as csvfile:  # open file
        csvreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        
        info = {}
        
        # Go through all kvr's
        for kvr in csvreader:
            if len(kvr) in info:
                info[len(kvr)] += 1
            else:
                info[len(kvr)] = 1
        
        print(info)
        
def normalize_ministerie(ministerie):
    # 10 gekozen, we returnen stam die in ministerie kan voorkomen (kunnen meerdere zijn maar kiezen eerste)
    try:
        for mini in _ministeries:
            if  mini in ministerie:
                return mini

    except Exception as e:
        # bij deze set zijn er een stuk of 20 niet te classificeren
        print('Error normalizeren', e, ministerie)
    return 'foutief'
        
def index_dataset():
    print('Reading in file')
    tic = time.time()
    
    # Read CSV file
    kvrdf = pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz', 
                   compression='gzip', sep='\t', 
                   index_col=0, names=_names,
                      )
    
    print('Finished reading file in: ', time.time() - tic)
    
    kvrdf.ministerie = kvrdf.ministerie.str.lower().apply(normalize_ministerie)
    
    for mini in _ministeries:
        # Get all data per ministry
        miniRows = kvrdf.loc[kvrdf['ministerie'] == mini]
        
        # Bag all text together
        text = '\n'.join(list(miniRows.titel)).join(list(miniRows.vraag)).join(miniRows.partij).join(miniRows.antwoord)
        BoW= Counter(nltk.word_tokenize(text.lower()))
        #TODO : clean data (stopwords)
        
        _index_dataset[mini] = BoW
#         for term, freq in BoW:
#             _index_dataset[mini][ = freq 
        
    
#     for index, row in kvrdf.iterrows():
#         text = nltk.word_tokenize(join(row['vraag'],row['partij'],row['titel'],row['antwoord']).lower())
#         ministerie = row['ministerie']
        
        
#         for term in text:
#             if ministerie in _index_dataset['classes']:
#                 _index_dataset['classes'][ministerie].append(kvrName)
#             else:
#                 _index_dataset['classes'][kvrMinistry] = []
#                 _index_dataset['classes'][kvrMinistry].append(kvrName)

#             # Go through every term in text
#             for t in text:
#                 # Check if term exists
#                 if t in _index_dataset['terms']:
#                     if kvrName in _index_dataset['terms'][t]:
#                         _index_dataset['terms'][t][kvrName] += 1
#                     else:
#                         _index_dataset['terms'][t][kvrName] = 1
#                 else:
#                     _index_dataset['terms'][t] = {}
#                     _index_dataset['terms'][t][kvrName] = 1
            
#     print(kvrdf.ministerie.unique())

index_dataset()

Reading in file
Finished reading file in:  0.7674276828765869
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float' is not iterable nan
Error normalizeren argument of type 'float

In [ ]:
print(_index_dataset.keys())

# Some example code

In [ ]:
import pandas as pd

names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

# Change to KVR1000.csv.gz if this becomes to0 slow for you
kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz', 
                   compression='gzip', sep='\t', 
                   index_col=0, names=names,
                   ) 

for kolom in names[1:]:
    kvrdf[kolom]= kvrdf[kolom].astype(str)
print(kvrdf.shape)
kvrdf.head()

In [ ]:
# want to know how this works?
pd.read_csv?

In [ ]:
kvrdf.ministerie.value_counts().head(20)

In [ ]:
import nltk
from collections import Counter

# Join all titels of the questions together
allvragen= '\n'.join(list(kvrdf.titel))
print(len(allvragen), allvragen[:1000])

In [ ]:
# Tokenize, lowercase  and turn into a bag of words
BoW= Counter(nltk.word_tokenize(allvragen.lower()))
print(len(BoW))
# show the top 20 most used words 
BoW.most_common(20)

In [ ]:
# remove stopwords, and tokens containing punctuation or numbers (note also removes "u2", "can't", etc)
from nltk.corpus import stopwords
DutchStop= stopwords.words('dutch')
print(len(DutchStop))

BoW=  Counter([w for w in nltk.word_tokenize(allvragen.lower()) if w.isalpha() and not w in set(DutchStop)])
print(len(BoW))
# show the top 20 most used words 
BoW.most_common(20)

In [ ]:
# how many words occur just once?
# how many occur at least 5 times?